In [1]:
# As discussed in the discussion section, we have leak in the competition if we use external data.
# D[](http://)iscussed here: https://www.kaggle.com/c/ga-customer-revenue-prediction/discussion/68235#401950
 
#  I have downloaded the data and used it to climb to the 2nd position. Before using it I was at the 600th position. 
#  Most of the code used it from olivier's (https://www.kaggle.com/ogrellier) notebooks in this competition and I haven't done any feature engineering with the external data yet.
 
#  You can use the data that I downloaded as it would be in the data of this kernel. 
#  There are 4 files 2 for train and 2 for test.
#Can't upload the notebook as kaggle kernels keep on crashing but this script will get you same score as mine.

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import os
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

import gc
import lightgbm as lgb
import xgboost as xgb
import catboost
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold

pd.set_option("display.max_rows", 1000, "display.max_columns", 1000)
pd.options.mode.chained_assignment = None

#using https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook
def load_df(csv_path='./train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [2]:
#Getting VisitId to Join with our train, test data
def get_visitid(id):
    bef_, af_ = str(x).split('.')
    return int(bef_), (int(af_)*10 if len(af_)==1 else int(af_))

def get_folds(df=None, n_splits=5):
    """Returns dataframe indices corresponding to Visitors Group KFold"""
    # Get sorted unique visitors
    unique_vis = np.array(sorted(df['fullVisitorId'].unique()))

    # Get folds
    folds = GroupKFold(n_splits=n_splits)
    fold_ids = []
    ids = np.arange(df.shape[0])
    for trn_vis, val_vis in folds.split(X=unique_vis, y=unique_vis, groups=unique_vis):
        fold_ids.append(
            [
                ids[df['fullVisitorId'].isin(unique_vis[trn_vis])],
                ids[df['fullVisitorId'].isin(unique_vis[val_vis])]
            ]
        )

    return fold_ids

def process_device(data_df):
    data_df['source.country'] = data_df['source'] + '_' + data_df['country']
    #data_df['campaign.medium'] = data_df['campaign'] + '_' + data_df['medium']
    data_df['browser.category'] = data_df['browser'] + '_' + data_df['deviceCategory']
    data_df['browser.os'] = data_df['browser'] + '_' + data_df['operatingSystem']
    
    return data_df


def custom(data):
    data['device_deviceCategory_channelGrouping'] = data['deviceCategory'] + "_" + data['channelGrouping']
    data['channelGrouping_browser'] = data['browser'] + "_" + data['channelGrouping']
    data['channelGrouping_OS'] = data['operatingSystem'] + "_" + data['channelGrouping']
    
    for i in ['city', 'continent', 'country','metro', 'networkDomain', 'region','subContinent']:
        for j in ['browser','deviceCategory', 'operatingSystem', 'source']:
            data[i + "_" + j] = data[i] + "_" + data[j]
    
    data['content.source'] = data['adContent'] + "_" + data['country']
    data['medium.source'] = data['medium'] + "_" + data['country']
    return data

def browser_mapping(x):
    browsers = ['chrome','safari','firefox','internet explorer','edge','opera','coc coc','maxthon','iron']
    if x in browsers:
        return x.lower()
    elif  ('android' in x) or ('samsung' in x) or ('mini' in x) or ('iphone' in x) or ('in-app' in x) or ('playstation' in x):
        return 'mobile browser'
    elif  ('mozilla' in x) or ('chrome' in x) or ('blackberry' in x) or ('nokia' in x) or ('browser' in x) or ('amazon' in x):
        return 'mobile browser'
    elif  ('lunascape' in x) or ('netscape' in x) or ('blackberry' in x) or ('konqueror' in x) or ('puffin' in x) or ('amazon' in x):
        return 'mobile browser'
    elif '(not set)' in x:
        return x
    else:
        return 'others'
    
def adcontents_mapping(x):
    if  ('google' in x):
        return 'google'
    elif  ('placement' in x) | ('placememnt' in x):
        return 'placement'
    elif '(not set)' in x or 'nan' in x:
        return x
    elif 'ad' in x:
        return 'ad'
    else:
        return 'others'
    
def source_mapping(x):
    if  ('google' in x):
        return 'google'
    elif  ('youtube' in x):
        return 'youtube'
    elif '(not set)' in x or 'nan' in x:
        return x
    elif 'yahoo' in x:
        return 'yahoo'
    elif 'facebook' in x:
        return 'facebook'
    elif 'reddit' in x:
        return 'reddit'
    elif 'bing' in x:
        return 'bing'
    elif 'quora' in x:
        return 'quora'
    elif 'outlook' in x:
        return 'outlook'
    elif 'linkedin' in x:
        return 'linkedin'
    elif 'pinterest' in x:
        return 'pinterest'
    elif 'ask' in x:
        return 'ask'
    elif 'siliconvalley' in x:
        return 'siliconvalley'
    elif 'lunametrics' in x:
        return 'lunametrics'
    elif 'amazon' in x:
        return 'amazon'
    elif 'mysearch' in x:
        return 'mysearch'
    elif 'qiita' in x:
        return 'qiita'
    elif 'messenger' in x:
        return 'messenger'
    elif 'twitter' in x:
        return 'twitter'
    elif 't.co' in x:
        return 't.co'
    elif 'vk.com' in x:
        return 'vk.com'
    elif 'search' in x:
        return 'search'
    elif 'edu' in x:
        return 'edu'
    elif 'mail' in x:
        return 'mail'
    elif 'ad' in x:
        return 'ad'
    elif 'golang' in x:
        return 'golang'
    elif 'direct' in x:
        return 'direct'
    elif 'dealspotr' in x:
        return 'dealspotr'
    elif 'sashihara' in x:
        return 'sashihara'
    elif 'phandroid' in x:
        return 'phandroid'
    elif 'baidu' in x:
        return 'baidu'
    elif 'mdn' in x:
        return 'mdn'
    elif 'duckduckgo' in x:
        return 'duckduckgo'
    elif 'seroundtable' in x:
        return 'seroundtable'
    elif 'metrics' in x:
        return 'metrics'
    elif 'sogou' in x:
        return 'sogou'
    elif 'businessinsider' in x:
        return 'businessinsider'
    elif 'github' in x:
        return 'github'
    elif 'gophergala' in x:
        return 'gophergala'
    elif 'yandex' in x:
        return 'yandex'
    elif 'msn' in x:
        return 'msn'
    elif 'dfa' in x:
        return 'dfa'
    elif '(not set)' in x:
        return '(not set)'
    elif 'feedly' in x:
        return 'feedly'
    elif 'arstechnica' in x:
        return 'arstechnica'
    elif 'squishable' in x:
        return 'squishable'
    elif 'flipboard' in x:
        return 'flipboard'
    elif 't-online.de' in x:
        return 't-online.de'
    elif 'sm.cn' in x:
        return 'sm.cn'
    elif 'wow' in x:
        return 'wow'
    elif 'baidu' in x:
        return 'baidu'
    elif 'partners' in x:
        return 'partners'
    else:
        return 'others'

In [3]:
PATH = '/media/axel/ssd/google-analytics/'
train = pd.read_feather(PATH + 'train')
test = pd.read_feather(PATH + 'test')

# train = load_df()
# test = load_df(PATH + "test.csv")

#Loading external data
train_store_1 = pd.read_csv('./Train_external_data.csv', low_memory=False, skiprows=6, dtype={"Client Id":'str'})
train_store_2 = pd.read_csv('./Train_external_data_2.csv', low_memory=False, skiprows=6, dtype={"Client Id":'str'})
test_store_1 = pd.read_csv('./Test_external_data.csv', low_memory=False, skiprows=6, dtype={"Client Id":'str'})
test_store_2 = pd.read_csv('./Test_external_data_2.csv', low_memory=False, skiprows=6, dtype={"Client Id":'str'})


for df in [train_store_1, train_store_2, test_store_1, test_store_2]:
    df["visitId"] = df["Client Id"].apply(lambda x: x.split('.', 1)[1]).astype(str)

train_exdata = pd.concat([train_store_1, train_store_2], sort=False)
test_exdata = pd.concat([test_store_1, test_store_2], sort=False)

for df in [train, test]:
    df["visitId"] = df["visitId"].astype(str)

# Merge with train/test data
train_new = train.merge(train_exdata, how="left", on="visitId")
test_new = test.merge(test_exdata, how="left", on="visitId")

# Drop Client Id
for df in [train_new, test_new]:
    df.drop("Client Id", 1, inplace=True)

#Cleaning Revenue
for df in [train_new, test_new]:
    df["Revenue"].fillna('$', inplace=True)
    df["Revenue"] = df["Revenue"].apply(lambda x: x.replace('$', '').replace(',', ''))
    df["Revenue"] = pd.to_numeric(df["Revenue"], errors="coerce")
    df["Revenue"].fillna(0.0, inplace=True)

#Imputing NaN
for df in [train_new, test_new]:
    df["Sessions"] = df["Sessions"].fillna(0)
    df["Avg. Session Duration"] = df["Avg. Session Duration"].fillna(0)
    df["Bounce Rate"] = df["Bounce Rate"].fillna(0)
    df["Revenue"] = df["Revenue"].fillna(0)
    df["Transactions"] = df["Transactions"].fillna(0)
    df["Goal Conversion Rate"] = df["Goal Conversion Rate"].fillna(0)
    df['adContent'].fillna('N/A', inplace=True)
    #df['slot'].fillna('N/A', inplace=True)
    #df['page'].fillna(0.0, inplace=True)
    #df['isVideoAd'].fillna('N/A', inplace=True)
    #df['adNetworkType'].fillna('N/A', inplace=True)
    #df['gclId'].fillna('N/A', inplace=True)
    df['isTrueDirect'].fillna('N/A', inplace=True)
    df['referralPath'].fillna('N/A', inplace=True)
    df['keyword'].fillna('N/A', inplace=True)
    #df['bounces'].fillna(0.0, inplace=True)
    #df['newVisits'].fillna(0.0, inplace=True)
    df['pageviews'].fillna(0.0, inplace=True)

del train
del test
train = train_new
test = test_new

for df in [train, test]:
    df['date_new'] = pd.to_datetime(df['visitStartTime'], unit='s')
    df['date'] = df['date_new'].dt.date
    df['sess_date_dow'] = df['date_new'].dt.dayofweek
    df['sess_date_hours'] = df['date_new'].dt.hour
    df['sess_month'] = df['date_new'].dt.month

# Dropping Constant Columns

const_cols = [col for col in train.columns if len(train[col].unique())==1]

for df in [train, test]:
    df.drop(const_cols, 1, inplace=True)

train.drop('campaignCode', 1, inplace=True)


y_reg = train['transactionRevenue'].fillna(0)
del train['transactionRevenue']

if 'transactionRevenue' in test.columns:
    del test['transactionRevenue']

excluded_features = [
    'date', 'date_new', 'fullVisitorId', 'sessionId', 'transactionRevenue', 
    'visitId', 'visitStartTime'
]


# train['browser'] = train['browser'].map(lambda x:browser_mapping(str(x).lower())).astype('str')
# train['adContent'] = train['adContent'].map(lambda x:adcontents_mapping(str(x).lower())).astype('str')
# train['source'] = train['source'].map(lambda x:source_mapping(str(x).lower())).astype('str')
# test['browser'] = test['browser'].map(lambda x:browser_mapping(str(x).lower())).astype('str')
# test['adContent'] = test['adContent'].map(lambda x:adcontents_mapping(str(x).lower())).astype('str')
# test['source'] = test['source'].map(lambda x:source_mapping(str(x).lower())).astype('str')

# train = process_device(train)
# test = process_device(test)
# train = custom(train)
# test = custom(test)


categorical_features = [
    _f for _f in train.columns
    if (_f not in excluded_features) & (train[_f].dtype == 'object')
]

for f in categorical_features:
    train[f], indexer = pd.factorize(train[f])
    test[f] = indexer.get_indexer(test[f])

for df in [train, test]:
    df['hits/pageviews'] = (df["pageviews"]/(df["hits"])).apply(lambda x: 0 if np.isinf(x) else x)
    df['is_high_hits'] = np.logical_or(df["hits"]>4,df["pageviews"]>4).astype(np.int32)
    df["Revenue"] = np.log1p(df["Revenue"])
    df['hits'] = np.log1p(df['hits'])
#     df['next_session_2'] = (
#         df['date'] - df[['fullVisitorId', 'date']].groupby('fullVisitorId')['date'].shift(-1)
#     ).astype(np.int64) // 1e9 // 60 // 60
    

# train['fullVisitorId_hits_mean'] = train.groupby('fullVisitorId')['Revenue'].transform('mean')
# test['fullVisitorId_hits_mean'] = test.groupby('fullVisitorId')['Revenue'].transform('mean')


num_folds = 10
folds = get_folds(df=train, n_splits=num_folds)
y_reg = y_reg.astype(float)
train_features = [_f for _f in train.columns if _f not in excluded_features]
print(train_features)

importances = pd.DataFrame()
oof_reg_preds = np.zeros(train.shape[0])
sub_reg_preds = np.zeros(test.shape[0])
for fold_, (trn_, val_) in enumerate(folds):
    trn_x, trn_y = train[train_features].iloc[trn_], y_reg.iloc[trn_]
    val_x, val_y = train[train_features].iloc[val_], y_reg.iloc[val_]
        
    params = {}
    params["device"] = "gpu"
    
    reg = lgb.LGBMRegressor(**params,
        num_leaves=31,
        learning_rate=0.03,
        n_estimators=1000,
        subsample=.9,
        colsample_bytree=.9,
        random_state=1
    )
    reg.fit(
        trn_x, np.log1p(trn_y),
        eval_set=[(val_x, np.log1p(val_y))],
        early_stopping_rounds=100,
        verbose=100,
        eval_metric='rmse'
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = train_features
    imp_df['gain'] = reg.booster_.feature_importance(importance_type='gain')
    
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_reg_preds[val_] = reg.predict(val_x, num_iteration=reg.best_iteration_)
    oof_reg_preds[oof_reg_preds < 0] = 0
    _preds = reg.predict(test[train_features], num_iteration=reg.best_iteration_)
    _preds[_preds < 0] = 0
    sub_reg_preds += np.expm1(_preds) / len(folds)
    
print(mean_squared_error(np.log1p(y_reg), oof_reg_preds) ** .5)

/home/axel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:120: RuntimeWarning: divide by zero encountered in log1p


['channelGrouping', 'visitNumber', 'browser', 'deviceCategory', 'isMobile', 'operatingSystem', 'city', 'continent', 'country', 'metro', 'networkDomain', 'region', 'subContinent', 'hits', 'pageviews', 'adContent', 'isTrueDirect', 'keyword', 'medium', 'referralPath', 'source', 'Sessions', 'Avg. Session Duration', 'Bounce Rate', 'Revenue', 'Transactions', 'Goal Conversion Rate', 'sess_date_dow', 'sess_date_hours', 'sess_month', 'hits/pageviews', 'is_high_hits', 'fullVisitorId_hits_mean']
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.5793
[200]	valid_0's rmse: 1.55979
[300]	valid_0's rmse: 1.55439
[400]	valid_0's rmse: 1.55081
[500]	valid_0's rmse: 1.54908
[600]	valid_0's rmse: 1.54802
[700]	valid_0's rmse: 1.54782
[800]	valid_0's rmse: 1.54852
Early stopping, best iteration is:
[733]	valid_0's rmse: 1.54768
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.59148
[200]	valid_0's rmse: 1.57931
[300]	valid_0's rmse

In [4]:
train['predictions'] = np.expm1(oof_reg_preds)
test['predictions'] = sub_reg_preds

# Aggregate data at User level
trn_data = train[train_features + ['fullVisitorId']].groupby('fullVisitorId').mean()

# Create a list of predictions for each Visitor
trn_pred_list = train[['fullVisitorId', 'predictions']].groupby('fullVisitorId')\
    .apply(lambda df: list(df.predictions))\
    .apply(lambda x: {'pred_'+str(i): pred for i, pred in enumerate(x)})

# Create a DataFrame with VisitorId as index
# trn_pred_list contains dict 
# so creating a dataframe from it will expand dict values into columns
trn_all_predictions = pd.DataFrame(list(trn_pred_list.values), index=trn_data.index)
trn_feats = trn_all_predictions.columns
trn_all_predictions['t_mean'] = np.log1p(trn_all_predictions[trn_feats].mean(axis=1))
trn_all_predictions['t_median'] = np.log1p(trn_all_predictions[trn_feats].median(axis=1))
trn_all_predictions['t_sum_log'] = np.log1p(trn_all_predictions[trn_feats]).sum(axis=1)
trn_all_predictions['t_sum_act'] = np.log1p(trn_all_predictions[trn_feats].fillna(0).sum(axis=1))
trn_all_predictions['t_nb_sess'] = trn_all_predictions[trn_feats].isnull().sum(axis=1)
full_data = pd.concat([trn_data, trn_all_predictions], axis=1)
del trn_data, trn_all_predictions
gc.collect()
print(full_data.shape)


sub_pred_list = test[['fullVisitorId', 'predictions']].groupby('fullVisitorId')\
    .apply(lambda df: list(df.predictions))\
    .apply(lambda x: {'pred_'+str(i): pred for i, pred in enumerate(x)})

sub_data = test[train_features + ['fullVisitorId']].groupby('fullVisitorId').mean()
sub_all_predictions = pd.DataFrame(list(sub_pred_list.values), index=sub_data.index)
for f in trn_feats:
    if f not in sub_all_predictions.columns:
        sub_all_predictions[f] = np.nan
sub_all_predictions['t_mean'] = np.log1p(sub_all_predictions[trn_feats].mean(axis=1))
sub_all_predictions['t_median'] = np.log1p(sub_all_predictions[trn_feats].median(axis=1))
sub_all_predictions['t_sum_log'] = np.log1p(sub_all_predictions[trn_feats]).sum(axis=1)
sub_all_predictions['t_sum_act'] = np.log1p(sub_all_predictions[trn_feats].fillna(0).sum(axis=1))
sub_all_predictions['t_nb_sess'] = sub_all_predictions[trn_feats].isnull().sum(axis=1)
sub_full_data = pd.concat([sub_data, sub_all_predictions], axis=1)
del sub_data, sub_all_predictions
gc.collect()
print(sub_full_data.shape)

# Create target at Visitor level
train['target'] = y_reg
trn_user_target = train[['fullVisitorId', 'target']].groupby('fullVisitorId').sum()

# Train a model at Visitor level
folds = get_folds(df=full_data[['pageviews']].reset_index(), n_splits=num_folds)

oof_preds = np.zeros(full_data.shape[0])
sub_preds = np.zeros(sub_full_data.shape[0])
vis_importances = pd.DataFrame()

for fold_, (trn_, val_) in enumerate(folds):
    trn_x, trn_y = full_data.iloc[trn_], trn_user_target['target'].iloc[trn_]
    val_x, val_y = full_data.iloc[val_], trn_user_target['target'].iloc[val_]
        
    params = {}
    params["device"] = "gpu"
    
    reg = lgb.LGBMRegressor(**params,
        num_leaves=31,
        learning_rate=0.03,
        n_estimators=1000,
        subsample=.9,
        colsample_bytree=.9,
        random_state=1
    )
    reg.fit(
        trn_x, np.log1p(trn_y),
        eval_set=[(trn_x, np.log1p(trn_y)), (val_x, np.log1p(val_y))],
        eval_names=['TRAIN', 'VALID'],
        early_stopping_rounds=100,
        eval_metric='rmse',
        verbose=100
    )
    
    imp_df = pd.DataFrame()
    imp_df['feature'] = trn_x.columns
    imp_df['gain'] = reg.booster_.feature_importance(importance_type='gain')
    
    imp_df['fold'] = fold_ + 1
    vis_importances = pd.concat([vis_importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_] = reg.predict(val_x, num_iteration=reg.best_iteration_)
    oof_preds[oof_preds < 0] = 0
    
    # Make sure features are in the same order
    _preds = reg.predict(sub_full_data[full_data.columns], num_iteration=reg.best_iteration_)
    _preds[_preds < 0] = 0
    sub_preds += _preds / len(folds)
    
print(mean_squared_error(np.log1p(trn_user_target['target']), oof_preds) ** .5)

# # Display feature importances
# vis_importances['gain_log'] = np.log1p(vis_importances['gain'])
# mean_gain = vis_importances[['gain', 'feature']].groupby('feature').mean()
# vis_importances['mean_gain'] = vis_importances['feature'].map(mean_gain['gain'])

# plt.figure(figsize=(8, 25))
# sns.barplot(x='gain_log', y='feature', data=vis_importances.sort_values('mean_gain', ascending=False).iloc[:300])

/home/axel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in log1p


(714167, 316)


/home/axel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in log1p


(617242, 316)
Training until validation scores don't improve for 100 rounds.
[100]	TRAIN's rmse: 1.45006	VALID's rmse: 1.46301
[200]	TRAIN's rmse: 1.41547	VALID's rmse: 1.45826
Early stopping, best iteration is:
[174]	TRAIN's rmse: 1.42154	VALID's rmse: 1.4579
Training until validation scores don't improve for 100 rounds.
[100]	TRAIN's rmse: 1.444	VALID's rmse: 1.507
[200]	TRAIN's rmse: 1.40897	VALID's rmse: 1.50285
[300]	TRAIN's rmse: 1.38828	VALID's rmse: 1.50297
Early stopping, best iteration is:
[220]	TRAIN's rmse: 1.40423	VALID's rmse: 1.50253
Training until validation scores don't improve for 100 rounds.
[100]	TRAIN's rmse: 1.44511	VALID's rmse: 1.495
[200]	TRAIN's rmse: 1.40964	VALID's rmse: 1.49223
Early stopping, best iteration is:
[164]	TRAIN's rmse: 1.41922	VALID's rmse: 1.49199
Training until validation scores don't improve for 100 rounds.
[100]	TRAIN's rmse: 1.44916	VALID's rmse: 1.46352
[200]	TRAIN's rmse: 1.41409	VALID's rmse: 1.45963
[300]	TRAIN's rmse: 1.39375	VALID's 

In [5]:
sub_full_data['PredictedLogRevenue'] = sub_preds
sub_full_data[['PredictedLogRevenue']].to_csv('../../submission.csv', index=True)